In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**This is a test for running Stable Diffusion (adapted to NovelAI model), created by physics1024 2022.10.11 10:09**

test GPUs

In [ ]:
!nvidia-smi

get Stable Diffusion webui version

In [ ]:
!mkdir ~/content
%cd ~/content

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

get model files

In [ ]:
!mkdir -p ~/content/stable-diffusion-webui/models/Stable-diffusion ~/content/stable-diffusion-webui/models/hypernetworks 
%cd ~/content/stable-diffusion-webui/models/Stable-diffusion

!apt install -y aria2

# Models

# 7G animefull-final-latest (optional)
!aria2c -d ~/content/stable-diffusion-webui https://cloudflare-ipfs.com/ipfs/bafybeiav3j7npiuewbel3mi32l3sidgkw54kuleosbhxmdvddbnvtfi7yu/config.yaml
!aria2c https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.ckpt

# 4G animefull-final-pruned 
# !aria2c animefull-final-pruned.ckpt https://cloudflare-ipfs.com/ipfs/bafybeicpamreyp2bsocyk3hpxr7ixb2g2rnrequub3j2ahrkdxbvfbvjc4/model.ckpt
# !aria2c -d ~/content/stable-diffusion-webui/config.yaml https://cloudflare-ipfs.com/ipfs/bafybeiav3j7npiuewbel3mi32l3sidgkw54kuleosbhxmdvddbnvtfi7yu/config.yaml

# Others

# Install embeddings (optional)
!aria2c https://cloudflare-ipfs.com/ipfs/bafybeie3hdjchxs5tz4n75bos53nhcklslguxchdurc2ynrzcfv2kwyklu/embeddings.tar
!tar xvf embeddings.tar -C ~/content/stable-diffusion-webui/embeddings && rm -rf embeddings.tar

# Install VAE Weights (optional)
!aria2c -o animefull-final-pruned.vae.pt https://cloudflare-ipfs.com/ipfs/bafybeiccldswdd3wvg57jhclcq53lvsc6gizasiblwayvhlv6eq4wow7wu/animevae.pt 
!ln -s animefull-final-pruned.vae.pt animefull-final-latest.vae.pt

# Install hypernetwork （optional)
!aria2c https://cloudflare-ipfs.com/ipfs/bafybeiduanx2b3mcvxlwr66igcwnpfmk3nc3qgxlpwh6oq6m6pxii3f77e/_modules.tar  
!tar xvf _modules.tar -C ~/content/stable-diffusion-webui/models/hypernetworks && rm -rf _modules.tar

load Stable Diffusion and models

In [ ]:
%cd ~/content/stable-diffusion-webui

# hacks to load model in kaggle
!sed -i 's/map_location="cpu"/map_location="cuda"/g' ~/content/stable-diffusion-webui/modules/sd_models.py

# Use novelai stable-diffusion-private (maybe slow, and the disk may not contain it, optional)
# !mkdir -p repositories/stable-diffusion
# !aria2c https://cloudflare-ipfs.com/ipfs/bafybeiauciguui6skzcwg23mjprjbuqhfzudms2gfi4rna55bygc3s6yxa/stable-diffusion-private.tar 
# !tar xvf stable-diffusion-private.tar -C repositories/stable-diffusion && rm -rf stable-diffusion-private.tar
# !git config --global --add safe.directory ~/content/stable-diffusion-webui/repositories/stable-diffusion
# !pip3 install icecream xformers
# !sed -i 's/transformer\.text_model/transformer/g' modules/sd_hijack.py
# !COMMANDLINE_ARGS="--share --gradio-debug --config config.yaml --xformers" \
#  REQS_FILE="requirements.txt" STABLE_DIFFUSION_COMMIT_HASH="5d95f1742ab1831d99df91c1a469b2b56a61eebf" python launch.py

# ...Or use normal stable-diffusion
!COMMANDLINE_ARGS="--share --gradio-debug --config config.yaml --disable-safe-unpickle" REQS_FILE="requirements.txt" python launch.py

pack images, please run the cell manually

In [11]:
# Pack images to output.zip. You can then download it at "Data-output" area.
!zip -q -r output.zip ~/content/stable-diffusion-webui/outputs 
!mv ~/content/stable-diffusion-webui/output.zip /kaggle/working/output.zip